In [1]:
# Import dependencies
import bit
from bit.network import NetworkAPI
import lit
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
import subprocess
import json
import os
from dotenv import load_dotenv
from constants import *

In [2]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [3]:
# Connects to private ETH POA blockchain
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8181'))
w3.isConnected()

False

In [4]:
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [14]:
# Create a function called `derive_wallets`
def derive_wallets(coin):
    cols = 'address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub'
    command = f'derive --mnemonic=mnemonic --cols={cols} --coin={coin} --numderive=2 --format=json -g'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [15]:
coins = {BTCTEST: derive_wallets(BTCTEST), ETH: derive_wallets(ETH)}

In [16]:
coins

{'btc-test': [{'address': 'mtgb9hthmFpY1zidrYmZ24eTF87aBA147h',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe',
   'pubkey': '0339b281b12687c70a024ef409229b55d404533b4e9a475dc9d3883c0172913fb6',
   'pubkeyhash': '906c250da82c2bd3edf9bcbbbb29f2188065f9d2',
   'xprv': 'tprv8kmimjp9GYq9EvcrxozEhxzk9jGwaC73BSiy3ATqwwikM4YgH7685zQLzX7xzXAmhXxf5LZbpbqtSGX3fcNFbfVJdxuEYPrA5tDRRQgwPRm',
   'xpub': 'tpubDHTkv9rPQvWp8PeerTeq7NeriknsjXHwkkKkKgW9NDX9BYoSuVuiGV2DAejLNv85zVm31akUTVk6Z8MyhECbWDzQ3Qir4GoRCXfBgKk8GxA'},
  {'address': 'mxwSU31v1NtbUi6Pc5bN9BgmmAGw13JdvX',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cRG5vrAkkq3ZkTmT2mmqCgbxd8niBdXx3ocDMd1771oSzJDvqXiC',
   'pubkey': '02f4468eae06b195a4f83cc63d680d015d512af60288977e0283fd4299bfcde24b',
   'pubkeyhash': 'bf1ba366dd65a4824c22f497bbff1fae89007841',
   'xprv': 'tprv8kmimjp9GYq9FvSyaAKU2MY38GBpiMHEsjgMtnLaMhyZQPzAiEM6ZTC8z9vdTsJVRQDcoCvLeWEHiPWWLpiGMjoYcffLhXJXN

In [7]:
# coins = {BTCTEST: derive_wallets(BTCTEST), ETH: derive_wallets(ETH), BTC: derive_wallets(BTC), LTC: derive_wallets(LTC)}

In [8]:
eth0_pk = coins['eth'][0]['privkey']
eth1_pk = coins['eth'][1]['privkey']
eth2_pk = coins['eth'][2]['privkey']
btctest0_pk = coins[BTCTEST][0]['privkey']
btctest1_pk = coins[BTCTEST][1]['privkey']
btctest2_pk = coins[BTCTEST][2]['privkey']

In [9]:
# Checks which coin then returns address from supplied private key
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return bit.PrivateKeyTestnet(priv_key)
#     elif coin == LTC:
#         return lit.PrivateKeyTestnet(priv_key)

In [10]:
priv_key_to_account(ETH, eth1_pk)

In [11]:
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        account = priv_key_to_account(coin, account)
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "chainId": 888,
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }, account
    elif coin == BTCTEST:
        account = priv_key_to_account(coin, account)
        return bit.PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)]), account

def send_tx(coin, account, recipient, amount):
    tx, account = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result
    elif coin == BTCTEST:
        result = NetworkAPI.broadcast_tx_testnet(signed_tx)
        return result

In [12]:
send_tx(ETH, eth0_pk, '0x926841419b49910D80fCEa75d587BE3aefFf560e', Web3.toWei(10, 'ether'))

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8181): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fafa7239ac0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
send_tx(ETH, coins[ETH][0]['privkey'], coins[ETH][2]['address'], Web3.toWei(1, 'ether'))

In [ ]:
send_tx(BTCTEST, coins[BTCTEST][0]['privkey'], coins[BTCTEST][2]['address'], .0001)

In [ ]:
coins